# Linear Regression

The following code uses Tensowflow's Estimator API to train a linear regression model. Code adapted from Tensorflow's linear regression tutorial: https://github.com/tensorflow/docs/blob/master/site/en/tutorials/estimator/linear.ipynb.

In [8]:
from itertools import combinations
import numpy as np
import os, sys
import pandas as pd
from pathlib import Path
from pi_pact_sort import categorize
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc

DROP_COLUMNS = ['ADDRESS', 'TIMESTAMP', 'UUID', 'MAJOR', 'MINOR', 'TX POWER', 'TEMPERATURE',
                'PITCH', 'ROLL', 'YAW', 'SCAN']
FEATURE_COLUMNS = ['RSSI', 'HUMIDITY', 'PRESSURE']
SAMPLE_SIZE = 30000

# Initialize DataFrame
data: pd.DataFrame = pd.DataFrame(columns=['RSSI', 'DISTANCE', 'HUMIDITY', 'PRESSURE'])
data_copy: pd.DataFrame = data.copy()
csv_file: Path
for csv_file in Path('.').glob('indoor-noObstruct-SenseHat*/*.csv'):
    datapart: pd.DataFrame = pd.read_csv(csv_file)
    for column in DROP_COLUMNS:
        if column in datapart.columns:
            datapart = datapart.drop([column], 1)
    data_copy = data_copy.append(datapart)

# Categorize distance
data_copy['DISTANCE'] = data_copy['DISTANCE'].map(categorize)

# Sample data from each distance category
for value in data_copy['DISTANCE'].unique():
    datapart = data_copy[data_copy.DISTANCE == value]
    datapart = datapart.sample(SAMPLE_SIZE, random_state=1)
    data = data.append(datapart)

# Assign features and labels
# min_max_scaler = preprocessing.MinMaxScaler()
# X: pd.DataFrame = pd.DataFrame(min_max_scaler.fit_transform(data),
#                                columns=['RSSI', 'DISTANCE', 'HUMIDITY', 'PRESSURE']).astype({'RSSI': 'float32',
#                                                                                              'DISTANCE': 'float32',
#                                                                                              'HUMIDITY': 'float32',
#                                                                                              'PRESSURE': 'float32'
#                                                                                              })
X: pd.DataFrame = data.astype({'RSSI': 'int32', 'DISTANCE': 'int32',
                                  'HUMIDITY': 'float32', 'PRESSURE': 'float32'})
y: pd.Series = X.pop('DISTANCE')
    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(y_train)
# Code below adapted from Tensorflow's "Build a linear model with Estimators" tutorial:
# https://github.com/tensorflow/docs/blob/master/site/en/tutorials/estimator/linear.ipynb
feature_columns = []
for feature_name in FEATURE_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

for feature1_name, feature2_name in combinations(FEATURE_COLUMNS, 2):
    feature_columns.append(tf.feature_column.crossed_column([feature1_name, feature2_name], hash_bucket_size=1000))

def make_input_fn(data_df, label_df, num_epochs=100, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn = make_input_fn(X_train, y_train)
eval_input_fn = make_input_fn(X_test, y_test, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns, model_dir=str(Path('./linear-models')),
                                           n_classes=3)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

211    0
138    1
61     2
342    2
358    1
      ..
299    0
186    1
451    1
446    2
280    1
Name: DISTANCE, Length: 81000, dtype: int32
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '.', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_rep

INFO:tensorflow:loss = 0.25947106, step = 6800 (0.919 sec)
INFO:tensorflow:global_step/sec: 136.604
INFO:tensorflow:loss = 0.3377052, step = 6900 (0.731 sec)
INFO:tensorflow:global_step/sec: 150.101
INFO:tensorflow:loss = 0.27457, step = 7000 (0.666 sec)
INFO:tensorflow:global_step/sec: 150.101
INFO:tensorflow:loss = 0.3690101, step = 7100 (0.666 sec)
INFO:tensorflow:global_step/sec: 151.233
INFO:tensorflow:loss = 0.14204174, step = 7200 (0.661 sec)
INFO:tensorflow:global_step/sec: 151.461
INFO:tensorflow:loss = 0.3720508, step = 7300 (0.660 sec)
INFO:tensorflow:global_step/sec: 150.552
INFO:tensorflow:loss = 0.5606983, step = 7400 (0.663 sec)
INFO:tensorflow:global_step/sec: 150.552
INFO:tensorflow:loss = 0.42339656, step = 7500 (0.664 sec)
INFO:tensorflow:global_step/sec: 150.778
INFO:tensorflow:loss = 0.15567234, step = 7600 (0.664 sec)
INFO:tensorflow:global_step/sec: 151.461
INFO:tensorflow:loss = 0.10080381, step = 7700 (0.660 sec)
INFO:tensorflow:global_step/sec: 151.691
INFO:te

INFO:tensorflow:loss = 0.26793176, step = 15000 (0.660 sec)
INFO:tensorflow:global_step/sec: 150.552
INFO:tensorflow:loss = 0.139445, step = 15100 (0.664 sec)
INFO:tensorflow:global_step/sec: 150.326
INFO:tensorflow:loss = 0.73606586, step = 15200 (0.665 sec)
INFO:tensorflow:global_step/sec: 148.544
INFO:tensorflow:loss = 0.4774235, step = 15300 (0.673 sec)
INFO:tensorflow:global_step/sec: 145.315
INFO:tensorflow:loss = 0.26207268, step = 15400 (0.688 sec)
INFO:tensorflow:global_step/sec: 146.805
INFO:tensorflow:loss = 0.13303828, step = 15500 (0.680 sec)
INFO:tensorflow:global_step/sec: 149.653
INFO:tensorflow:loss = 0.16808164, step = 15600 (0.669 sec)
INFO:tensorflow:global_step/sec: 149.877
INFO:tensorflow:loss = 0.15295543, step = 15700 (0.667 sec)
INFO:tensorflow:global_step/sec: 150.326
INFO:tensorflow:loss = 0.40383154, step = 15800 (0.665 sec)
INFO:tensorflow:global_step/sec: 150.326
INFO:tensorflow:loss = 0.39489722, step = 15900 (0.665 sec)
INFO:tensorflow:global_step/sec: 1

INFO:tensorflow:global_step/sec: 147.236
INFO:tensorflow:loss = 0.15577462, step = 23200 (0.679 sec)
INFO:tensorflow:global_step/sec: 132.981
INFO:tensorflow:loss = 0.3861115, step = 23300 (0.754 sec)
INFO:tensorflow:global_step/sec: 102.314
INFO:tensorflow:loss = 0.1053029, step = 23400 (0.975 sec)
INFO:tensorflow:global_step/sec: 132.629
INFO:tensorflow:loss = 0.21353148, step = 23500 (0.754 sec)
INFO:tensorflow:global_step/sec: 142.022
INFO:tensorflow:loss = 0.10568036, step = 23600 (0.704 sec)
INFO:tensorflow:global_step/sec: 142.224
INFO:tensorflow:loss = 0.18921494, step = 23700 (0.703 sec)
INFO:tensorflow:global_step/sec: 142.628
INFO:tensorflow:loss = 0.43906736, step = 23800 (0.701 sec)
INFO:tensorflow:global_step/sec: 141.222
INFO:tensorflow:loss = 0.36579755, step = 23900 (0.708 sec)
INFO:tensorflow:global_step/sec: 141.621
INFO:tensorflow:loss = 0.2973006, step = 24000 (0.706 sec)
INFO:tensorflow:global_step/sec: 140.975
INFO:tensorflow:loss = 0.12129034, step = 24100 (0.70

INFO:tensorflow:loss = 0.23279348, step = 31300 (0.665 sec)
INFO:tensorflow:global_step/sec: 149.43
INFO:tensorflow:loss = 0.42079353, step = 31400 (0.668 sec)
INFO:tensorflow:global_step/sec: 151.233
INFO:tensorflow:loss = 0.24245961, step = 31500 (0.662 sec)
INFO:tensorflow:global_step/sec: 149.43
INFO:tensorflow:loss = 0.16739167, step = 31600 (0.669 sec)
INFO:tensorflow:global_step/sec: 150.552
INFO:tensorflow:loss = 0.27941805, step = 31700 (0.664 sec)
INFO:tensorflow:global_step/sec: 149.43
INFO:tensorflow:loss = 0.37897494, step = 31800 (0.669 sec)
INFO:tensorflow:global_step/sec: 149.653
INFO:tensorflow:loss = 0.43368816, step = 31900 (0.668 sec)
INFO:tensorflow:global_step/sec: 151.005
INFO:tensorflow:loss = 0.34271282, step = 32000 (0.662 sec)
INFO:tensorflow:global_step/sec: 149.877
INFO:tensorflow:loss = 0.22363108, step = 32100 (0.667 sec)
INFO:tensorflow:global_step/sec: 150.552
INFO:tensorflow:loss = 0.44159204, step = 32200 (0.664 sec)
INFO:tensorflow:global_step/sec: 1

...

INFO:tensorflow:global_step/sec: 103.218
INFO:tensorflow:loss = 0.20018524, step = 217500 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.059
INFO:tensorflow:loss = 0.07557582, step = 217600 (0.970 sec)
INFO:tensorflow:global_step/sec: 103.156
INFO:tensorflow:loss = 0.19814867, step = 217700 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.312
INFO:tensorflow:loss = 0.22622328, step = 217800 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.045
INFO:tensorflow:loss = 0.2672215, step = 217900 (0.970 sec)
INFO:tensorflow:global_step/sec: 104.108
INFO:tensorflow:loss = 0.09513598, step = 218000 (0.962 sec)
INFO:tensorflow:global_step/sec: 93.272
INFO:tensorflow:loss = 0.22175062, step = 218100 (1.071 sec)
INFO:tensorflow:global_step/sec: 82.3941
INFO:tensorflow:loss = 0.2251778, step = 218200 (1.214 sec)
INFO:tensorflow:global_step/sec: 84.8975
INFO:tensorflow:loss = 0.20087563, step = 218300 (1.177 sec)
INFO:tensorflow:global_step/sec: 98.6269
INFO:tensorflow:loss = 0.2446982, step = 218

INFO:tensorflow:global_step/sec: 86.9969
INFO:tensorflow:loss = 0.12699407, step = 225600 (1.147 sec)
INFO:tensorflow:global_step/sec: 123.69
INFO:tensorflow:loss = 0.31867248, step = 225700 (0.808 sec)
INFO:tensorflow:global_step/sec: 113.014
INFO:tensorflow:loss = 0.19591856, step = 225800 (0.885 sec)
INFO:tensorflow:global_step/sec: 129.224
INFO:tensorflow:loss = 0.40676162, step = 225900 (0.774 sec)
INFO:tensorflow:global_step/sec: 132.891
INFO:tensorflow:loss = 0.08007394, step = 226000 (0.753 sec)
INFO:tensorflow:global_step/sec: 118.011
INFO:tensorflow:loss = 0.16364387, step = 226100 (0.847 sec)
INFO:tensorflow:global_step/sec: 106.215
INFO:tensorflow:loss = 0.22701584, step = 226200 (0.942 sec)
INFO:tensorflow:global_step/sec: 106.585
INFO:tensorflow:loss = 0.22220948, step = 226300 (0.936 sec)
INFO:tensorflow:global_step/sec: 105.945
INFO:tensorflow:loss = 0.11635142, step = 226400 (0.944 sec)
INFO:tensorflow:global_step/sec: 105.307
INFO:tensorflow:loss = 0.1512803, step = 2

INFO:tensorflow:global_step/sec: 134.95
INFO:tensorflow:loss = 0.1518971, step = 233700 (0.740 sec)
INFO:tensorflow:global_step/sec: 116.048
INFO:tensorflow:loss = 0.127727, step = 233800 (0.863 sec)
INFO:tensorflow:global_step/sec: 106.948
INFO:tensorflow:loss = 0.19335996, step = 233900 (0.935 sec)
INFO:tensorflow:global_step/sec: 107.847
INFO:tensorflow:loss = 0.36999094, step = 234000 (0.926 sec)
INFO:tensorflow:global_step/sec: 105.901
INFO:tensorflow:loss = 0.11270724, step = 234100 (0.945 sec)
INFO:tensorflow:global_step/sec: 101.793
INFO:tensorflow:loss = 0.20693535, step = 234200 (0.982 sec)
INFO:tensorflow:global_step/sec: 106.958
INFO:tensorflow:loss = 0.16789898, step = 234300 (0.935 sec)
INFO:tensorflow:global_step/sec: 105.614
INFO:tensorflow:loss = 0.18346193, step = 234400 (0.948 sec)
INFO:tensorflow:global_step/sec: 104.445
INFO:tensorflow:loss = 0.29202467, step = 234500 (0.956 sec)
INFO:tensorflow:global_step/sec: 107.114
INFO:tensorflow:loss = 0.2782749, step = 2346

INFO:tensorflow:global_step/sec: 134.166
INFO:tensorflow:loss = 0.29994395, step = 241800 (0.746 sec)
INFO:tensorflow:global_step/sec: 134.856
INFO:tensorflow:loss = 0.27678907, step = 241900 (0.742 sec)
INFO:tensorflow:global_step/sec: 136.315
INFO:tensorflow:loss = 0.21010372, step = 242000 (0.733 sec)
INFO:tensorflow:global_step/sec: 134.88
INFO:tensorflow:loss = 0.23750113, step = 242100 (0.741 sec)
INFO:tensorflow:global_step/sec: 133.836
INFO:tensorflow:loss = 0.18303129, step = 242200 (0.747 sec)
INFO:tensorflow:global_step/sec: 134.632
INFO:tensorflow:loss = 0.19058737, step = 242300 (0.743 sec)
INFO:tensorflow:global_step/sec: 133.512
INFO:tensorflow:loss = 0.120232865, step = 242400 (0.750 sec)
INFO:tensorflow:global_step/sec: 133.833
INFO:tensorflow:loss = 0.30773973, step = 242500 (0.746 sec)
INFO:tensorflow:global_step/sec: 110.971
INFO:tensorflow:loss = 0.33212662, step = 242600 (0.902 sec)
INFO:tensorflow:global_step/sec: 106.996
INFO:tensorflow:loss = 0.11191354, step =

INFO:tensorflow:loss = 0.24421391, step = 249600 (0.816 sec)
INFO:tensorflow:global_step/sec: 133.378
INFO:tensorflow:loss = 0.50614226, step = 249700 (0.750 sec)
INFO:tensorflow:global_step/sec: 134.562
INFO:tensorflow:loss = 0.24037595, step = 249800 (0.744 sec)
INFO:tensorflow:global_step/sec: 135.68
INFO:tensorflow:loss = 0.1226052, step = 249900 (0.735 sec)
INFO:tensorflow:global_step/sec: 135.147
INFO:tensorflow:loss = 0.13926443, step = 250000 (0.741 sec)
INFO:tensorflow:global_step/sec: 134.857
INFO:tensorflow:loss = 0.23070139, step = 250100 (0.741 sec)
INFO:tensorflow:global_step/sec: 134.853
INFO:tensorflow:loss = 0.17341352, step = 250200 (0.742 sec)
INFO:tensorflow:global_step/sec: 133.905
INFO:tensorflow:loss = 0.038570765, step = 250300 (0.747 sec)
INFO:tensorflow:global_step/sec: 131.24
INFO:tensorflow:loss = 0.21110329, step = 250400 (0.762 sec)
INFO:tensorflow:global_step/sec: 83.5201
INFO:tensorflow:loss = 0.40145636, step = 250500 (1.199 sec)
INFO:tensorflow:global_

In [9]:
print(result)

{'accuracy': 0.9021111, 'average_loss': 0.22840096, 'loss': 0.22952048, 'global_step': 253200}
